In [1]:
import numpy as np                                      #import required libraries 
from matplotlib import *
from math import *
from sympy import *
import matplotlib.pyplot as plt

In [2]:
class NACAXXXX:                                          # class for NACA airfoil to get x and y coordinates of airfoil
    
    def __init__(self, digits, c):
        self.digits = str(digits)
        self.c = c                                                                 # replace x by x/c for scaling to the chord length
        self.m = float(self.digits[0])*(.01)*c                                       # max camber, m = (first digit from left)* (c/100)
        self.p = float(self.digits[1])*(.1)*c                                        # pos_max camber, p = (2nd digit from left)* (c/10)
        self.tmax = float(self.digits[2:])*(.01)*c                                   # max thickness, tmax = (last two digit)* (c/100)
        
    def camber(self, x):
        m = self.m
        p = self.p
        c = self.c

        if p == 0:
            yc = m*(c-x)*(1 + x/c - 2*p)/((1-p)**2)
        
        else:

            if 0 < x <= p*c :
                yc = (m/(p*p))*(2*p*x - x*x/c)
                
            else:
                yc = m*(c-x)*(1 + x/c - 2*p)/((1-p)**2)
            
        print("camber z coordinates: ", yc)
        return yc
            
    def thickness(self, x):
        tmax = self.tmax
        c = self.c
        #yt = ((tmax*c)/0.2)*(0.2969*sqrt(x/c) - 0.1260*(x/c) - 0.3516*pow(x/c, 2) + 0.2843*pow(x/c, 3) - 0.1015*pow(x/c, 4))
        yt = (5*tmax*c)*(0.2969*(x/c)**0.5-0.126*(x/c)-0.3516*(x/c)**2+0.2843*(x/c)**3-0.1036*(x/c)**4)
        print("thickness-camber to boundary length: ", yt)
        return yt
        
    def ftheta(self, x):
        m = self.m
        p = self.p
        c = self.c

        if p == 0:
            if x < 1e-6:
                theta1 = 0
            else:
                z = symbols('z')
                yc = m*(c-z)*(1 + z/c - 2*p)/((1-p*p)**2)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #yc_ = diff(yc, z)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)

        else:
            if 0 <= x <= p:
                z = symbols('z')
                yc = (m*z/(p*p))*(2*p - z/c)
                #yc_ = diff(yc, z)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)

            elif p < x <= c:
                z = symbols('z')
                yc = m*(c-z)*(1 + z/c - 2*p)/((1-p*p)**2)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #yc_ = diff(yc, z)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)
        
        #print("theta1 is: ", theta1)
        return theta1      

    
    def get_Xu(self, x):
        self.Xu = x - self.thickness(x) * np.sin(self.ftheta(x))
        return self.Xu
    
    def get_Zu(self, x):
        #self.Zu = self.camber(x) + self.thickness(x) * np.cos(self.ftheta(x))
        self.Zu = self.camber(x) + self.thickness(x)
        #print("value of Zu is: ", self.Zu)
        return self.Zu
    
    def get_Xl(self, x):
        self.Xl = x + self.thickness(x) * np.sin(self.ftheta(x))
        return self.Xl
    
    def get_Zl(self, x):
        #self.Zl = self.camber(x) - self.thickness(x) * np.cos(self.ftheta(x))
        self.Zl = self.camber(x) - self.thickness(x) 
        #print("value of Zl is: ", self.Zl)
        return self.Zl



In [3]:
class vortex_lattice_method:
    
    def __init__(self, m, alpha, c, Zu, Zl):
        self.m = m                                      # m = no of panels, resulting into m+1 nodes
        self.alpha = alpha                              # alpha = angle of attack of airfoil
        self.c = c                                      # c = camber length
        self.Zu = Zu                                    # Zu = Z coordinates of upper part of airfoil
        self.Zl = Zl                                    # Zl = Z coordinates of lower part of airfoil
        
        self.X_j = np.zeros(m+1)                                 # X_j = X coordinates of jth node
        self.Z_j = np.zeros(m+1)                                 # Z_j = Z coordinates of jth node
        self.node_coordinates()
        
        self.X__i = np.zeros(m)                                  # X__i = X coordinates of control point of ith panel
        self.Z__i = np.zeros(m)                                  # Z__i = Z coordinate of control point of ith panel
        self.control_coordinates()
        
        self.l__i = np.zeros(m)
        self.length__i = np.zeros(m)
        self.length_panel()
        
        self.theta__i = np.zeros(m)
        self.theta_panel()
        
        self.c_ij__n1=np.zeros((m,m))                                           
        self.c_ij__n2=np.zeros((m,m))                                           
        self.c_ij__t1=np.zeros((m,m))
        self.c_ij__t2=np.zeros((m,m))
        self.normal_tangential_influence_coefficients()
        
        
        self.matrix_W_ij = np.zeros((m+1, m+1))
        self.matrix_C_i = np.zeros(m+1)
        self.gamma = np.zeros(m+1)
        self.solution_Y()

        self.n__i=np.zeros((m,2))                                               #2-D, (m rows & 2 columns ) array of zeros for unit normals
        self.t__i=np.zeros((m,2))                                               #2-D, (m rows & 2 columns ) array of zeros for unit tangents
        self.normal_tangent_panel()
        
        
    def node_coordinates(self):                               # function to get coordinates of jth node
        m = self.m 
        c = self.c
        Zu = self.Zu
        Zl = self.Zl
        X_j = self.X_j
        Z_j = self.Z_j

        #theta = np.linspace(-1,1,m+1)*np.pi                       
 
        #lower_nodes=0                                                           #nodes of lower surfaces
        #for i in range(0,theta.size):
            #if theta[i]<=0:
                #lower_nodes+=1                                                  #determinig the number of lower nodes
    
        #X_j = 0.5*(1 - np.cos(theta))                                          #glauert form
 
        #for j in range(0,lower_nodes):
            #Z_j[j] = Zl(X_j[j])                                                #lower surface nodes
 
        #for j in range(lower_nodes,m+1): 
            #Z_j[j] = Zu(X_j[j])

        theta = np.linspace(2*np.pi, 0, m+1)

        for j in range(0, theta.size):

            X_j[j] = 0.5*c*(1+np.cos(theta[j]))         # m+1-j because X[j] is taken in anticlock direction

        for j in range(0, theta.size):

            if np.pi < theta[j] <= 2*np.pi:
                Z_j[j] = Zl(X_j[j])

            else:
                Z_j[j] = Zu(X_j[j])                                             #upper surface nodes

        self.X_j = X_j
        self.Z_j = Z_j
        
        print("division of circle: ", theta)
        print("X node coordinate is : ", self.X_j)
        print("Z node coordinate is : ", self.Z_j)

    def control_coordinates(self):                            # function to get coordinates of control point of ith panel
        m = self.m
        X_j = self.X_j
        Z_j = self.Z_j
        X__i = self.X__i
        Z__i = self.Z__i

        for i in range(0, m):
            X__i[i] = (X_j[i] + X_j[i+1])/2
            Z__i[i] = (Z_j[i] + Z_j[i+1])/2

        self.X__i = X__i
        self.Z__i = Z__i


    def length_panel(self):
        m = self.m
        length__i = self.length__i
        X_j = self.X_j
        Z_j = self.Z_j

        for i in range(0, m):
            length__i[i] = ((X_j[i+1]-X_j[i])**2 + (Z_j[i+1]-Z_j[i])**2)**0.5
            
        self.length__i = length__i
        print("length of panel: ", self.length__i)

    def theta_panel(self):
        m = self.m
        X_j = self.X_j
        Z_j = self.Z_j
        theta__i = self.theta__i

        for i in range(0, m):
            theta__i[i] = np.arctan2((Z_j[i+1]-Z_j[i]), (X_j[i+1]-X_j[i]))

        self.theta__i = theta__i
        print("theta_panel is : ", self.theta__i)

    def normal_tangent_panel(self):
        m=self.m
        theta__i=self.theta__i
        X_j=self.X_j
        Z_j=self.Z_j
        n__i=self.n__i
        t__i=self.t__i

        for i in range(0,m):                                                    #n=-sin(theta)i+cos(theta)k
            n__i[i][0]=-np.sin(theta__i[i])  
            n__i[i][1]=np.cos(theta__i[i])   
 
        for i in range(0,m):                                                    #t= cos(theta)i+sin(theta)k
            t__i[i][0]=np.cos(theta__i[i])
            t__i[i][1]=np.sin(theta__i[i])
        
        self.n__i=n__i
        self.t__i=t__i


    def normal_tangential_influence_coefficients(self):

        X_j = self.X_j
        Z_j = self.Z_j
        X__i = self.X__i
        Z__i = self.Z__i
        theta__i = self.theta__i
        length__i = self.length__i
        c_ij__n1 = self.c_ij__n1                                                  
        c_ij__n2 = self.c_ij__n2                                                  
        c_ij__t1 = self.c_ij__t1                                                  
        c_ij__t2 = self.c_ij__t2

        for i in range(0, m):
            for j in range(0, m):
                if i == j:
                    c_ij__n1[i][j] = -1
                    c_ij__n2[i][j] = 1
                    c_ij__t1[i][j] = np.pi/2
                    c_ij__t2[i][j] = np.pi/2
                else:
                    A = -((X__i[i]-X_j[j])*np.cos(theta__i[j])) - ((Z__i[i]-Z_j[j])*np.sin(theta__i[j]))
                    B = (X__i[i]-X_j[j])**2 + (Z__i[i]-Z_j[j])**2
                    C = np.sin(theta__i[i] - theta__i[j])
                    D = np.cos(theta__i[i] - theta__i[j])
                    E = ((X__i[i]-X_j[j])*np.sin(theta__i[j])) - ((Z__i[i]-Z_j[j])*np.cos(theta__i[j]))
                    F = np.log(1+((length__i[j]**2)+(2*length__i[j]*A))/B)
                    G = np.arctan2((E*length__i[j]), (B+A*length__i[j]))
                    P= (((X__i[i]-X_j[j])*(np.sin(theta__i[i]-2*theta__i[j])))+((Z__i[i]-Z_j[j])*(np.cos(theta__i[i]-2*theta__i[j]))))
                    Q= (((X__i[i]-X_j[j])*(np.cos(theta__i[i]-2*theta__i[j])))-((Z__i[i]-Z_j[j])*(np.sin(theta__i[i]-2*theta__i[j]))))

                    c_ij__n2[i][j]= D + 0.5*Q*F/(length__i[j]) - (A*C+D*E)*G/(length__i[j])   
                    c_ij__n1[i][j]= 0.5*D*F + C*G - c_ij__n2[i][j]              

                    c_ij__t2[i][j]= C + (0.5*P*F)/length__i[j] + (A*D-C*E)*G/length__i[j] 
                    c_ij__t1[i][j]= 0.5*C*F - D*G - c_ij__t2[i][j]
    
        #print("value of coeff1:", c_ij__n1)
        #print("value of coeff2:", c_ij__n2)

        self.c_ij__n1=c_ij__n1
        self.c_ij__n2=c_ij__n2
        self.c_ij__t1=c_ij__t1
        self.c_ij__t2=c_ij__t2



    def solution_Y(self):
        m = self.m
        alpha = self.alpha
        X__i = self.X__i
        Z__i = self.Z__i
        theta__i = self.theta__i
        length__i = self.length__i
        c_ij__n1 = self.c_ij__n1
        c_ij__n2 = self.c_ij__n2
        c_ij__t1 = self.c_ij__t1
        c_ij__t2 = self.c_ij__t2
        matrix_W_ij = self.matrix_W_ij
        matrix_C_i = self.matrix_C_i
        gamma = self.gamma



        # write matrix W and C using for loop, then solve for Y

        # writing C matrix using for loop

        for i in range(0, m):
            matrix_C_i[i] = np.sin(theta__i[i]-alpha)

        matrix_C_i[m] = 0
        #print("jeeta matrix is: ", matrix_C_i)

        # writing W matrix using for loop

        for i in range(0, m):
            for j in range(1,m):
                matrix_W_ij[i][j] = c_ij__n2[i][j-1] + c_ij__n1[i][j]

            matrix_W_ij[i][0] = c_ij__n1[i][0]          # j = 0
            matrix_W_ij[i][m] = c_ij__n2[i][m-1]        # j = m

        #for j in range(1, m):
            #matrix_W_ij[m][j] = 0

        matrix_W_ij[m][0] = 1                           # i = m, j = 0
        matrix_W_ij[m][m] = 1                           # i = m, j = m

        print("influence coefficient matrix W is: ", matrix_W_ij)

        Y = np.linalg.solve(matrix_W_ij, matrix_C_i)
        print("circulation strength matrix T is: ", Y)
        gamma = Y


        self.matrix_W_ij = matrix_W_ij
        self.matrix_C_i = matrix_C_i
        self.gamma = gamma
        

    

In [4]:
digits = (input("enter the four digits of NACAXXXX airfoil: "))
c = float(input("enter the chord length: "))
b = float(input("enter the span length: "))
m = int(input("enter no of panels n: "))
alpha = float(input("enter angle of attack of wing: "))

airfoil_1 = NACAXXXX(digits, c)
result_vlm = vortex_lattice_method(m, alpha*(np.pi)/180, c, airfoil_1.get_Zu, airfoil_1.get_Zl)
result_vlm.solution_Y()

IndexError: string index out of range